# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
city_df= pd.read_csv("city_weather.csv")
del city_df["Unnamed: 0"]
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,cape town,0.0,ZA,1.650939e+09,73.0,-33.9258,18.4232,60.91,11.99
1,ushuaia,40.0,AR,1.650939e+09,70.0,-54.8000,-68.3000,40.66,16.11
2,bengkulu,100.0,ID,1.650939e+09,74.0,-3.8004,102.2655,80.01,3.83
3,soyo,68.0,AO,1.650939e+09,90.0,-6.1349,12.3689,77.18,5.97
4,nikolskoye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Handle NaN values
city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)
# Locations
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]


In [18]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# # Narrow DataFrame to find better weather condition.
narrowed_city_df = city_df

# # A max temperature lower than 80 degrees but higher than 70.
narrowed_city_df = narrowed_city_df.loc[(narrowed_city_df["Max Temp"] < 80) & (narrowed_city_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
narrowed_city_df= narrowed_city_df.loc[narrowed_city_df["Windspeed"] < 10]

# # Zero cloudiness.
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Cloudiness"] == 0]

# # Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
20,caravelas,0.0,BR,1.650939e+09,83.0,-17.7125,-39.2481,75.54,4.03
22,oak ridge,0.0,US,1.650939e+09,72.0,36.0104,-84.2696,72.25,0.00
32,saint george,0.0,US,1.650939e+09,10.0,37.1041,-113.5841,75.79,4.61
132,cabo san lucas,0.0,MX,1.650940e+09,44.0,22.8909,-109.9124,79.11,5.75
224,iranshahr,0.0,IR,1.650940e+09,29.0,27.2025,60.6848,75.54,4.61
303,vila velha,0.0,BR,1.650940e+09,84.0,-20.3297,-40.2925,74.95,1.14
419,seoul,0.0,KR,1.650940e+09,53.0,37.5683,126.9778,76.57,6.91
498,san vicente,0.0,AR,1.650940e+09,81.0,-26.6167,-54.1333,70.21,5.12
500,fairhope,0.0,US,1.650940e+09,68.0,30.5230,-87.9033,73.67,0.00
584,taoudenni,0.0,ML,1.650940e+09,26.0,22.6783,-3.9836,73.24,8.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
20,caravelas,BR,-17.7125,-39.2481,
22,oak ridge,US,36.0104,-84.2696,
32,saint george,US,37.1041,-113.5841,
132,cabo san lucas,MX,22.8909,-109.9124,
224,iranshahr,IR,27.2025,60.6848,
303,vila velha,BR,-20.3297,-40.2925,
419,seoul,KR,37.5683,126.9778,
498,san vicente,AR,-26.6167,-54.1333,
500,fairhope,US,30.5230,-87.9033,
584,taoudenni,ML,22.6783,-3.9836,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [22]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
     # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    # Print end of search once searching is completed
print("-------Data Retrieval Complete-------")

Retrieving Results for Index 20: caravelas.
Closest hotel in caravelas is Pousada dos Navegantes.
------------
Retrieving Results for Index 22: oak ridge.
Closest hotel in oak ridge is TownePlace Suites by Marriott Knoxville Oak Ridge.
------------
Retrieving Results for Index 32: saint george.
Closest hotel in saint george is Hilton Garden Inn St. George.
------------
Retrieving Results for Index 132: cabo san lucas.
Closest hotel in cabo san lucas is Hotel Riu Palace Cabo San Lucas.
------------
Retrieving Results for Index 224: iranshahr.
Closest hotel in iranshahr is مهمانسرا گاز.
------------
Retrieving Results for Index 303: vila velha.
Closest hotel in vila velha is Hotel Santorini.
------------
Retrieving Results for Index 419: seoul.
Closest hotel in seoul is The Westin Josun Seoul.
------------
Retrieving Results for Index 498: san vicente.
Closest hotel in san vicente is Hotel Y Restaurante Los Lapachos.
------------
Retrieving Results for Index 500: fairhope.
Closest hotel 

In [23]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
20,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes
22,oak ridge,US,36.0104,-84.2696,TownePlace Suites by Marriott Knoxville Oak Ridge
32,saint george,US,37.1041,-113.5841,Hilton Garden Inn St. George
132,cabo san lucas,MX,22.8909,-109.9124,Hotel Riu Palace Cabo San Lucas
224,iranshahr,IR,27.2025,60.6848,مهمانسرا گاز
303,vila velha,BR,-20.3297,-40.2925,Hotel Santorini
419,seoul,KR,37.5683,126.9778,The Westin Josun Seoul
498,san vicente,AR,-26.6167,-54.1333,Hotel Y Restaurante Los Lapachos
500,fairhope,US,30.5230,-87.9033,"The Grand Hotel Golf Resort & Spa, Autograph C..."
584,taoudenni,ML,22.6783,-3.9836,


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))